# Investigate twitter API v.2 
- most useful guide i could find: https://www.youtube.com/watch?v=0EekpQBEP_8&t=2s


- I have included notes from some of the most important topics in the video. 
    -  No notes included from 35mins onward (small functions)




In [3]:
import tweepy
import config #file containing API KEYS

In [6]:
#setup API instance
api = tweepy.Client(bearer_token=config.bearer_token, consumer_key=config.api_key, consumer_secret=config.api_key_secret, access_token=config.access_token, access_token_secret=config.access_token_secret)


#Construct query
query = '#nyancat OR #BAYC'
'''
the above query returns tweets that contain the term #nyancat or the term #BAYC

There are many other operators you can use for intelligent queries, like the above
    -Get only retweets by including "is:retweet"
    -Dont get any retweets by including "-is:retweet"
    -Get only tweets that contain media by including "has:media"
    
    -Much much more functionality. For example, can specify geolocation of tweet (ONLY ACADEMIC LICENSE)

There is a whole module about writing queries at https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research 
There is also documentation at: https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query
'''


'\nthe above query returns tweets that contain the term #nyancat or the term #BAYC\n\nThere are many other operators you can use for intelligent queries, like the above\n    -Get only retweets by including "is:retweet"\n    -Dont get any retweets by including "-is:retweet"\n    -Get only tweets that contain media by including "has:media"\n    \n    -Much much more functionality. For example, can specify geolocation of tweet (ONLY ACADEMIC LICENSE)\n\nThere is a whole module about writing queries at https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research \nThere is also documentation at: https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query\n'

**Search recent tweets (up to 7 days ago)**
- Only required parameter is query
- Specify max nr. of tweets to return with max_results. Default is 10, max is 100

In [7]:
response1 = api.search_recent_tweets(query=query, max_results=10)

for tweet in response1.data:
    print(tweet.id)

1497219600987324416
1497219598378143747
1497219558549110817
1497219551515263029
1497219510851588096
1497219505751220224
1497219462591827973
1497219446787940352
1497219317754118151
1497219273260863490


**How to get more data than just tweet ID and text**
- By deafult, tweets come with tweet id and text
- Use 'tweet_fields' and 'expansions' parameters to retrieve more info
    - Some tweet fields and expansions are restricted to academic license

In [11]:
#Tweet fields
'''
See possibilities for tweet fields at https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent#Optional
'''

tweet_fields=['text','created_at','lang','public_metrics']

response2 = api.search_recent_tweets(query=query, max_results=10, tweet_fields=tweet_fields)


for i in response2.data:
    print(i.text)
    print(i.lang)

RT @VandalizedNfts: #Giveaway 
3 WHITELIST SPOTS GIVEAWAY 🔥

✅ RT + JOIN DISCORD
https://t.co/NPnkaheKUn

⌛️Ends in 5h
#vandalized #nftart…
en
RT @tagholiday: #comics -&gt; #InuGames -&gt; #eSports -&gt; #NFTs -&gt; #P2E -&gt; #DEX -&gt;#Staking-&gt; #Records -&gt; @Disney -&gt; @MarvelStudios = $CPTINU 💎

💹…
en
RT @hoomaneth: 1 person among the first 6000 who retweet, follow me and turn on 🔔 will win @BoredApeYC MUTANT #24220 worth 18,3 ETH ($54,05…
en
RT @hoomaneth: 1 person among the first 6000 who retweet, follow me and turn on 🔔 will win @BoredApeYC MUTANT #24220 worth 18,3 ETH ($54,05…
en
RT @hoomaneth: 1 person among the first 6000 who retweet, follow me and turn on 🔔 will win @BoredApeYC MUTANT #24220 worth 18,3 ETH ($54,05…
en
RT @allfreeairdrop: FREE #NFT #sbayc GIVEAWAY 
Just drop your $ETH address, ❤️, RT &amp; Follow

Shadowfied #BAYC #7911👇👇👇 https://t.co/BQiaBiP…
en
RT @NFT_Rusty: @TurkeysNfts volume absolutely exploding the last 24 hours! Do not sleep on this unique pro

In [7]:
#Expansions
# Used to get even more data (e.g. user information)
    #for example, information about the tweet author is retrieved through the 'author_id' expansion
    #Expansion data is stored in reponse.includes

#each expansion includes multiple subgroups, and each subgroup contains multiple data points (fields).
    #Some fields are default, others have to be specified 
    #e.g. The 'users' subgroup of the author_id expansion includes the fiedlds id, name and username by default
    #request additional fields for an expansion subgroup with the fields parameter. 
    #For example request additional fields in the users subgroup with the user_fields parameter. One of the available fields is profile_image_url


tweet_fields=['text','created_at','lang','public_metrics']

expansions = ['author_id']
user_fields = ['profile_image_url']

response3 = api.search_recent_tweets(query=query, max_results=100, tweet_fields=tweet_fields, user_fields = user_fields, expansions=expansions)

users = {u['id']: u for u in response3.includes['users']}


for i in response3.data:
    if users[i.author_id]:
        user = users[i.author_id]
        print(i.id)
        print(user.profile_image_url)


BadRequest: 400 Bad Request
The `max_results` query parameter value [500] is not between 10 and 100

In [2]:
#geo.place_id is an expansion that gives you info about where the tweet is posted from


**How to get more than 100 tweets**
- Each API search request can retrieve a limited number of tweets (upper limit on max_tweets)
    - 100 for search_recent_tweets
    - 500 for search_all_tweets
- Can easily combine results from multiple requests by using tweepy paginator
    - For example, if want to retrieve 2000 tweets from search_all_tweets, we combine four requests of 500 each


In [35]:
for tweet in tweepy.Paginator(api.search_recent_tweets, query=query, max_results=100).flatten(limit=1000):
    print(tweet.id)

1497250553998372864
1497250552597549056
1497250530250293253
1497250416492519451
1497250386863788032
1497250323093766149
1497250293943132161
1497250270480420869
1497250249697640452
1497250189077356555
1497250176003711043
1497250175416475650
1497250143883915267
1497250137143484431
1497250136900214803
1497250122257899549
1497250093514346499
1497250066888855561
1497250055341830147
1497250019593932806
1497249962668826646
1497249937431556098
1497249924206845953
1497249850089345026
1497249849841963008
1497249835602386945
1497249821203304448
1497249791490748416
1497249773065314315
1497249761770053687
1497249722301653016
1497249715255394305
1497249707734847490
1497249705272623108
1497249704819761152
1497249687271010307
1497249684750016513
1497249646229561376
1497249562259771394
1497249524154507264
1497249520480133121
1497249488326598673
1497249454109466644
1497249398283243523
1497249372127608839
1497249323058348037
1497249259783028741
1497249222676021248
1497249215159930886
1497249196943949834


In [ ]:
tweets=[]
for tweet in tweepy.Paginator(client.search_all_tweets, query=query, max_results=500, start_time = start_time, end_time=end_time, tweet_fields = tweet_fields).flatten(limit=2000):
    tweets.append(tweet)

print(len(tweets))
print(tweets[0].text)
print(tweets[0].id)

**Full Archive Search**

- Only eligible under academic license
- Very similar as search_recent_tweets, but specify start_time and end_time

In [4]:
client = tweepy.Client(bearer_token=config.academic_bearer_token)

query = 'wordle -is:retweet -has:images lang:en'
start_time = '2022-01-25T00:00:00Z'
end_time = '2022-01-29T00:00:00Z'

tweet_fields=['text','created_at','lang','public_metrics']

response7 = client.search_all_tweets(query=query, max_results=500, start_time = start_time, end_time=end_time, tweet_fields = tweet_fields)

tweets = []
for tweet in response7.data:
    tweets.append(tweet)

In [5]:
import pandas as pd

datetimes = [tweet.created_at for tweet in tweets]
texts = [tweet.text for tweet in tweets]
retweet_counts = [tweet.public_metrics['retweet_count'] for tweet in tweets]
reply_counts = [tweet.public_metrics['reply_count'] for tweet in tweets]
like_counts = [tweet.public_metrics['like_count'] for tweet in tweets]

df = pd.DataFrame(data = {'datetime':datetimes, 'text':texts, 'retweet_count':retweet_counts, 'reply_count':reply_counts, 'like_count':like_counts})

df.head()
    

,datetime,text,retweet_count,reply_count,like_count
0,2022-01-28 23:59:59+00:00,Wordle 223 5/6\n\n⬛⬛⬛⬛⬛\n🟨🟩⬛⬛⬛\n⬛🟩🟩⬛⬛\n⬛🟩🟩🟩🟩\n...,0,0,1
1,2022-01-28 23:59:58+00:00,Wordle 224 5/6\n\n⬜🟨⬜⬜🟨\n🟨🟩⬜🟨⬜\n⬜🟩🟩⬜🟩\n⬜🟩🟩🟩🟩\n...,0,0,0
2,2022-01-28 23:59:58+00:00,Wordle 224 4/6*\n\n⬜🟨⬜⬜🟨\n⬜🟩🟩⬜🟩\n⬜🟩🟩🟩🟩\n🟩🟩🟩🟩🟩\...,0,1,1
3,2022-01-28 23:59:58+00:00,Wordle 224 5/6\n\n⬜🟩🟩⬜⬜\n⬜🟨🟨⬜⬜\n🟨🟩🟩⬜⬜\n⬜🟩🟩⬜🟩\n...,0,0,1
4,2022-01-28 23:59:57+00:00,@Sako_506 Wordle (ES) #22 2/6\n\n⬜⬜🟩🟨🟩\n🟩🟩🟩🟩🟩\...,0,1,0


In [6]:
df.to_csv('wordle_v1.csv', index=False)

**Getting tweet volume for a certain search query**
- Works for both recent search and full archive search 
- Splits into time chunks, specify size of chunks with granularity parameter


In [50]:
counts = api.get_recent_tweets_count(query='#wordle', granularity = 'day')


In [51]:
print(counts.data)

[{'end': '2022-02-19T00:00:00.000Z', 'start': '2022-02-18T17:18:22.000Z', 'tweet_count': 3655}, {'end': '2022-02-20T00:00:00.000Z', 'start': '2022-02-19T00:00:00.000Z', 'tweet_count': 14384}, {'end': '2022-02-21T00:00:00.000Z', 'start': '2022-02-20T00:00:00.000Z', 'tweet_count': 15548}, {'end': '2022-02-22T00:00:00.000Z', 'start': '2022-02-21T00:00:00.000Z', 'tweet_count': 13667}, {'end': '2022-02-23T00:00:00.000Z', 'start': '2022-02-22T00:00:00.000Z', 'tweet_count': 11509}, {'end': '2022-02-24T00:00:00.000Z', 'start': '2022-02-23T00:00:00.000Z', 'tweet_count': 10689}, {'end': '2022-02-25T00:00:00.000Z', 'start': '2022-02-24T00:00:00.000Z', 'tweet_count': 9400}, {'end': '2022-02-25T17:18:22.000Z', 'start': '2022-02-25T00:00:00.000Z', 'tweet_count': 7337}]


**Getting a user's timeline and mentions**


In [45]:
#first find userid for the user of interest
users = api.get_users(usernames=['twitterdev'])
for user in users:
    print(user)

#2244994945 


[<User id=2244994945 name=Twitter Dev username=TwitterDev>]
{}
[]
{}


In [47]:
#get users timeline
tweets = api.get_users_tweets(id=2244994945,tweet_fields=['lang'])

for tweet in tweets.data:
    print(tweet.text)
    print(tweet.lang)


Bay area, don't forget tomorrow is the first #TwitterDevConnect meetup of 2022! We have a few spots left so RSVP now: https://t.co/AahQG8MzCK https://t.co/5ah3tsCjNJ
en
Raw data to insights in a matter of minutes.⏱ Introducing the Twitter API toolkit for Google Cloud: access the brand new guide to easily process, analyze, and visualize massive amounts of Tweets today.👇 #BuildWhatsNext

https://t.co/jEXo0X6Fsp
en
Join @ashevat, @jessicagarson and @alanbenlee Thursday 2/24 at 3:05pm PT for this month’s town hall conversation on the recent updates to the Twitter Developer Platform. https://t.co/aJ7yayEwbx
en
We want to meet you! If you are in the Bay Area, join us for a #TwitterDev Connect meetup on February 24! RSVP here: https://t.co/AahQG8uYea
en
New ways to build discovery and analytics tools for #TwitterSpaces. Learn more about our new endpoint that returns Tweets from a Space, plus the new subcriber_count field. 👀  ⬇️  #BuildWhatsNext

https://t.co/Ev3Stajmjl
en
Celebrate the #GoodB

In [48]:
#get users mentions
tweets = api.get_users_mentions(id=2244994945,tweet_fields=['lang'])

for tweet in tweets.data:
    print(tweet.text)
    print(tweet.lang)

We created a series of videos for @TwitterDev highlighting powerful developer stories; introducing the people behind the code, showing the world anyone can be a developer with the Twitter API 🌎 https://t.co/xgIynaC73p
en
@JoselinMane @TwitterSpaces @MadalynSklar @BigTentSocial @chrismessina @MattNavarra @wongmjane @samsheffer @TwitterDev this looks ✨ !
en
@TwitterDev Hey folks, my twitter dev account was blocked without any notification of any violation, even when I did not use it for anything production , it was merely personal usage as developer ?
is there anyway of restoring my access even if you will remove all apps there ?
en
And why are "Pokemon" results popping up on the search list when you search "Wordle"?

@TwitterDev
en
給料月200万くらいもらってんでしょ？
なんでそんな仕事遅いの？ 

@twitter @twitterjp @TwitterDevJP @TwitterDev https://t.co/k53jamH9JC
ja
@TwitterDev
und
@Twitter @TwitterDev I could see now that has changed and it's uppercase 'M' with 1 minute and lowercase with &gt;1 minute. I think eve